In [1]:
import copy
import pandas
import warnings
import subprocess
from matplotlib import pyplot as plt
import os
import sys
from scipy import stats
import pprint
from IPython import display
import math

sys.path.insert(0, "../src")

from perf_tools.analysis import make_differential_frame, get_data, get_summary_statistics
from perf_tools.analysis import check_are_close, make_latency_plot, plot_latency_stats

csv_60 = "../datasets/genny2/perf_criteria_60.csv"
csv_70 = "../datasets/genny2/perf_criteria_70.csv"


/home/mark/.local/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# medical_workload_status_50_50_unencrypted
#medical_workload_credit_cards_100_0_unencrypted
#medical_workload_diagnosis_100_0
def parse_task_name(tn): 
    tn = tn.replace("credit_cards", "creditcards")
    p = tn.split("_")
    column = p[2]
    read = p[3]
    write = p[4]
    return (column, read, write)

print(parse_task_name("medical_workload_diagnosis_100_0"))
print(parse_task_name("medical_workload_credit_cards_100_0_unencrypted"))

print(parse_task_name("medical_workload_status_50_50_unencrypted"))


('diagnosis', '100', '0')
('creditcards', '100', '0')
('status', '50', '50')


In [3]:

# def extract_city_name(df):
#     """
#     Chicago, IL -> Chicago for city_name column
#     """
#     #df["city_name"] = df["city_and_code"].str.split(",").str.get(0)
#     pd["test_type"]  = pd["Task Name"].str.split("_").get(0)

#     return df

# medical_workload_status_50_50_unencrypted
#medical_workload_credit_cards_100_0_unencrypted
#medical_workload_diagnosis_100_0
def parse_task_name(tn): 
    tn = tn.replace("credit_cards", "creditcards")
    if tn == "medical_workload_load" or tn == "medical_workload_load_unencrypted":
        return ("all", 0, 0)

    p = tn.split("_")
    if len(p) < 5:
        raise ValueError(tn)
    column = p[2]
    read = p[3]
    write = p[4]
    return (column, read, write)

def get_test_type(t) :
    if "inserts" in t:
        return "inserts"
    if "update" in t:
        return "update"
    return "query"

def add_columns(df):
    # df = df.query(f"Test == 'InsertActor.load.inserts'")

    # df["test_type"] = df["Task Name"].apply(lambda x : x.split("_")[0])
    df["encrypted_type"] = df["Task Name"].apply(lambda x : "unencrypted" if "unencrypted" in x else "encrypted" )

    df["query_column"] = df["Task Name"].apply(lambda x : parse_task_name(x)[0] )

    df["read_percentage"] = df["Task Name"].apply(lambda x : parse_task_name(x)[1] )
    df["write_percentage"] = df["Task Name"].apply(lambda x : parse_task_name(x)[2] )

    df["test_type"] = df["Test"].apply(lambda x : get_test_type(x) )

    df["topo_type"] = df["Topology"].apply(lambda x : "replset" if "replSet" in x else "sharding" )
    return df


df_60 = pandas.read_csv(csv_60)
df_60 = df_60.query("Test != 'InsertActor.Crud'")
df_60["version"] = "60"
df_60 = add_columns(df_60)
df_70 = pandas.read_csv(csv_70)
df_70 = df_70.query("Test != 'InsertActor.Crud'")
df_70["version"] = "70"
df_70 = add_columns(df_70)

df = pandas.concat([df_60, df_70], ignore_index=True)

# a = pd["Task Name"].str.split("_").get(0)

#pd = pd.pipe(extract_city_name)
df.to_csv("/tmp/summary.csv")
# pd["AverageLatency"].plot()
df



,Patch ID,Execution,Task Name,Topology,Test,OperationsTotal,DocumentsTotal,SizeTotal,DurationTotal,OverheadTotal,...,WorkersMin,WorkersMax,ErrorRate,version,encrypted_type,query_column,read_percentage,write_percentage,test_type,topo_type
0,63fd86001e2d172b6c537ce4,0,medical_workload_credit_cards_100_0,linux-shard-lite-fle.2022-11,InsertActor.load.inserts,1000000,1000000,523715361,15559926000000,998616366933,...,16.0,16.0,0.0,60,encrypted,creditcards,100,0,inserts,sharding
1,63fd86001e2d172b6c537ce4,0,medical_workload_credit_cards_100_0,linux-shard-lite-fle.2022-11,InsertActor.FSM.update,0,0,0,0,0,...,NaN,NaN,NaN,60,encrypted,creditcards,100,0,update,sharding
2,63fd86001e2d172b6c537ce4,0,medical_workload_credit_cards_100_0,linux-shard-lite-fle.2022-11,InsertActor.FSM.query,100000,998960,744941581,916614000000,250013478915881,...,16.0,16.0,0.0,60,encrypted,creditcards,100,0,query,sharding
3,63fd86001e2d172b6c537ce4,0,medical_workload_credit_cards_100_0_unencrypted,linux-shard-lite-fle.2022-11,InsertActor.load.inserts,1000000,1000000,523707885,342025000000,1413365918133,...,16.0,16.0,0.0,60,unencrypted,creditcards,100,0,inserts,sharding
4,63fd86001e2d172b6c537ce4,0,medical_workload_credit_cards_100_0_unencrypted,linux-shard-lite-fle.2022-11,InsertActor.FSM.update,0,0,0,0,0,...,NaN,NaN,NaN,60,unencrypted,creditcards,100,0,update,sharding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,63fd867c3627e0496bc53086,0,medical_workload_status_50_50_unencrypted,linux-shard-lite-fle.2022-11,InsertActor.FSM.update,49785,47295,0,5321502000000,89424647970341,...,16.0,16.0,0.0,70,unencrypted,status,50,50,update,sharding
141,63fd867c3627e0496bc53086,0,medical_workload_status_50_50_unencrypted,linux-shard-lite-fle.2022-11,InsertActor.FSM.query,50215,2510758032,1314919069174,5320744000000,6672543137179,...,16.0,16.0,0.0,70,unencrypted,status,50,50,query,sharding
142,63fd867c3627e0496bc53086,0,medical_workload_status_95_5_unencrypted,linux-shard-lite-fle.2022-11,InsertActor.load.inserts,1000000,1000000,523717042,351806000000,1450414826135,...,16.0,16.0,0.0,70,unencrypted,status,95,5,inserts,sharding
143,63fd867c3627e0496bc53086,0,medical_workload_status_95_5_unencrypted,linux-shard-lite-fle.2022-11,InsertActor.FSM.update,5017,4776,0,9924215000000,163145385718286,...,16.0,16.0,0.0,70,unencrypted,status,95,5,update,sharding


In [4]:
def title_case(name):
    # type: (str) -> str
    """Return a CapitalCased version of a string."""

    # Only capitalize the last part of a fully-qualified name
    pos = name.rfind("::")
    if pos > -1:
        return name[:pos + 2] + name[pos + 2:pos + 3].upper() + name[pos + 3:]

    return name[0:1].upper() + name[1:]


In [5]:
df = pandas.DataFrame()

df[0]  = [1,2,3]

df[1] = [4,5,6]
df


,0,1
0,1,4
1,2,5
2,3,6


In [19]:

def get_numbers(topo, column, test, test_type, enc_type, version):
    query = f"topo_type == '{topo}' & query_column == '{column}' & read_percentage == {test[0]} & write_percentage == {test[1]} & test_type == '{test_type}' & encrypted_type == '{enc_type}' & version == {version}"
    rows = df.query(query)
    if rows.empty:
        # raise ValueError("No Rows: " + str(query))
        return ("Missing", "Missing")
    rc = rows.size / len(df.columns)
    if rc > 1:
        raise ValueError("Too many: %s - %s " % (rc, query))
    #assert row count
    r = rows

    ops_sec = rows["OperationThroughput"]
    p95_lat = rows["Latency95thPercentile"] / 1000000


    if  math.isnan(ops_sec) :
        raise ValueError("NAN is bad %s, %s - %s - %s" % (ops_sec, p95_lat, query , r))

    return (ops_sec.iloc[0], p95_lat.iloc[0])

# Bulk
# - 60 rs
# -- latency , ops_sec
# - 70 rs
# -- latency , ops_sec
# - 60 sharding
# -- latency , ops_sec
# - 70 sharding
# -- latency , ops_sec

rows = [
    ("creditcards", (50, 50)),
    ("diagnosis", (50, 50)),
    ("guid", (50, 50)),
    ("states", (50, 50)),
    ("status", (50, 50)),

    ("creditcards", (95, 5)),
    ("diagnosis", (95, 5)),
    ("guid", (95, 5)),
    ("states", (95, 5)),
    ("status", (95, 5)),

    ("creditcards", (100,0)),
    ("diagnosis", (100,0)),
    ("guid", (100,0)),
    ("states", (100,0)),
    ("status", (100,0)),

    ("load", (50, 50)),
]


In [21]:
df = pandas.read_csv("/tmp/summary.csv")

topo = "sharding"
dfr = None
for row in rows:
    query_cat = row[0]
    (read_percentage, write_percentage) = row[1]

    data = {}
    data["test"] = f"{read_percentage}% Read, {write_percentage}% Update"
    data["column"] = title_case(query_cat)

    for fsm_type in ["query", "update"]:
        q_type = "inserts" if query_cat == "load" else fsm_type
        if query_cat == "load" and fsm_type == "update":
            continue

        # No results for write_percentage == 0
        if write_percentage == 0 and fsm_type == "update":
            continue

        for version in [60, 70]:
            query_name = "creditcards" if query_cat == "load" else query_cat
            numbers = get_numbers(topo, query_name, row[1], q_type, "encrypted", version)

            (ops_sec, p95_lat) = numbers


            ops_quality = "unknown"
            latency_quality = "unknown"

            if ops_sec != "Missing":
                ops_sec = round(float(ops_sec))
                p95_lat = round(float(p95_lat))

                ops_quality = "bad"
                latency_quality = "bad"
                if query_cat == "load":
                    if ops_sec > 500:
                        ops_quality = "good"
                        latency_quality = "good"
                else:
                    if (read_percentage == 100 and p95_lat < 50) or \
                        (read_percentage == 95 and p95_lat < 55) or \
                        (read_percentage == 50 and p95_lat < 75):
                            ops_quality = "good"
                            latency_quality = "good"

            data["fsm"] = fsm_type

            data[f"{version}_p95"] = str(p95_lat) + "ms" if p95_lat != "Missing" else "Missing"
            data[f"{version}_ops"] = ops_sec

        sdf = pandas.DataFrame(index = [0], data=data)

        if dfr is None:
            dfr = sdf
        else:
            dfr = pandas.concat([dfr, sdf], ignore_index=True)

dfr.to_csv("/tmp/summary2.csv")
dfr

,test,column,fsm,60_p95,60_ops,70_p95,70_ops
0,"50% Read, 50% Update",Creditcards,query,173ms,45,22ms,204
1,"50% Read, 50% Update",Creditcards,update,238ms,44,36ms,202
2,"50% Read, 50% Update",Diagnosis,query,310ms,26,7ms,94
3,"50% Read, 50% Update",Diagnosis,update,365ms,26,31ms,93
4,"50% Read, 50% Update",Guid,query,71ms,95,3ms,510
5,"50% Read, 50% Update",Guid,update,147ms,94,37ms,506
6,"50% Read, 50% Update",States,query,Missing,Missing,Missing,Missing
7,"50% Read, 50% Update",States,update,Missing,Missing,Missing,Missing
8,"50% Read, 50% Update",Status,query,Missing,Missing,Missing,Missing
9,"50% Read, 50% Update",Status,update,Missing,Missing,Missing,Missing


In [22]:

output = """<html>
<style>
.good {
  background-color: lightgreen;
}
.bad {
  background-color: lightcoral;
}
</style>
<body><table>

<tr><th>Test</th>
<th>FSM</th>
<th colspan="4">1 Shard</th>
</tr>
<tr><th>Test</th>
<th>FSM</th>
<th colspan="2">MongoDB Non-Optimized</th>
<th colspan="2">Experimental 7.0</th>
</tr>

<tr><th>Test</th>
<th>FSM</th>
<th>Ops</th>
<th>P95</th>
<th>Ops</th>
<th>P95</th>
</tr>
"""

topo = "sharding"
for row in rows:

    for fsm_type in ["query", "update"]:
        query_cat = row[0]
        (read_percentage, write_percentage) = row[1]

        q_type = "inserts" if query_cat == "load" else fsm_type
        if query_cat == "load" and fsm_type == "update":
            continue

        # No results for write_percentage == 0
        if write_percentage == 0 and fsm_type == "update":
            continue


        output += "<tr>"
        output += f"<td>{title_case(query_cat)} {read_percentage}% Read, {write_percentage}% Update</td>"
        output += f"<td>{fsm_type}</td>"

        for version in [60, 70]:
            query_name = "creditcards" if query_cat == "load" else query_cat

            numbers = get_numbers(topo, query_name, row[1], q_type, "encrypted", version)
            #print(numbers)

            (ops_sec, p95_lat) = numbers

            ops_quality = "unknown"
            latency_quality = "unknown"

            if ops_sec != "Missing":
                ops_sec = round(float(ops_sec))
                p95_lat = round(float(p95_lat))

                ops_quality = "bad"
                latency_quality = "bad"
                if query_cat == "load":
                    if ops_sec > 500:
                        ops_quality = "good"
                        latency_quality = "good"
                else:
                    if (read_percentage == 100 and p95_lat < 50) or \
                        (read_percentage == 95 and p95_lat < 55) or \
                        (read_percentage == 50 and p95_lat < 75):
                            ops_quality = "good"
                            latency_quality = "good"

            output += f"<td class='{ops_quality}'>{ops_sec}</td><td class='{latency_quality}'>{p95_lat}</td>"
        output += "</tr>"

output += "</table></body></html>"

# output

display.HTML(output)

In [91]:
# df = pandas.read_csv("/tmp/summary.csv")

# def get_numbers(topo, column, test, test_type, enc_type, version):
#     query = f"topo_type == '{topo}' & query_column == '{column}' & read_percentage == {test[0]} & write_percentage == {test[1]} & test_type == '{test_type}' & encrypted_type == '{enc_type}' & version == {version}"
#     rows = df.query(query)
#     if rows.empty:
#         # raise ValueError("No Rows: " + str(query))
#         return ("Not Available", "Not Available")
#     rc = rows.size / len(df.columns)
#     if rc > 1:
#         raise ValueError("Too many: %s - %s " % (rc, query))
#     #assert row count
#     r = rows

#     ops_sec = rows["OperationThroughput"]
#     p95_lat = rows["Latency95thPercentile"] / 1000000

#     return (ops_sec.iloc[0], p95_lat.iloc[0])


# # Bulk
# # - 60 rs
# # -- latency , ops_sec
# # - 70 rs
# # -- latency , ops_sec
# # - 60 sharding
# # -- latency , ops_sec
# # - 70 sharding
# # -- latency , ops_sec

# rows = [
#     ("load", (50, 50)),
#     ("creditcards", (50, 50)),
#     ("diagnosis", (50, 50)),
#     ("guid", (50, 50)),
#     ("states", (50, 50)),
#     ("status", (50, 50)),
# ]

# output = """<html>
# <style>
# .good {
#   background-color: lightgreen;
# }
# .bad {
#   background-color: lightcoral;
# }
# </style>
# <body><table>

# <tr><th>Test</th>
# <th colspan="4">Replica Set</th>
# <th colspan="4">1 Shard</th>
# </tr>
# <tr><th>Test</th>
# <th colspan="2">MongoDB Non-Optimized</th>
# <th colspan="2">Experimental 7.0</th>
# <th colspan="2">MongoDB Non-Optimized</th>
# <th colspan="2">Experimental 7.0</th>
# </tr>
# <tr><th>Test</th>
# <th>Ops</th>
# <th>P95</th>
# <th>Ops</th>
# <th>P95</th>
# <th>Ops</th>
# <th>P95</th>
# <th>Ops</th>
# <th>P95</th>
# </tr>
# """

# for row in rows:
#     output += "<tr>"
#     query_cat = row[0]
#     (read_percentage, write_percentage) = row[1]
#     output += f"<td>{title_case(query_cat)} {read_percentage}% Read, {write_percentage}% Update</td>"
#     for topo in ["replset", "sharding"]:
#         for version in [60, 70]:
#             q_type = "inserts" if query_cat == "load" else "query"
#             query_name = "creditcards" if query_cat == "load" else query_cat
#             numbers = get_numbers(topo, query_name, row[1], q_type, "encrypted", version)
#             #print(numbers)

#             (ops_sec, p95_lat) = numbers

#             ops_quality = "unknown"
#             latency_quality = "unknown"

#             if ops_sec != "Not Available":
#                 ops_sec = round(float(ops_sec))
#                 p95_lat = round(float(p95_lat))

#                 ops_quality = "bad"
#                 latency_quality = "bad"
#                 if query_cat == "load":
#                     if ops_sec > 500:
#                         ops_quality = "good"
#                         latency_quality = "good"
#                 else:
#                     if (read_percentage == 100 and p95_lat < 50) or \
#                         (read_percentage == 95 and p95_lat < 55) or \
#                         (read_percentage == 50 and p95_lat < 75):
#                             ops_quality = "good"
#                             latency_quality = "good"

#             output += f"<td class='{ops_quality}'>{ops_sec}</td><td class='{latency_quality}'>{p95_lat}</td>"
#     output += "</tr>"

# output += "</table></body></html>"

# # output

# display.HTML(output)


In [42]:
# df = pandas.read_csv("/tmp/summary.csv")
# #df

# topos = ["replset", "sharding"]
# columns = [ 'creditcards', 'diagnosis', 'guid', 'states', 'status']
# # tests = [(0,0), (100,0), (95, 5), (50,50)]
# test_types = ["insert", "query"]
# tests = [(100,0), (95, 5), (50,50)]
# enc_types = ["unencrypted", "encrypted"]

# #df

# r = None

# enc_type = "encrypted"
# test_type = "query"
# version = 60
# for topo in topos:
#     for column in columns:
#         for test in tests:
#             query = f"topo_type == '{topo}' & query_column == '{column}' & read_percentage == {test[0]} & write_percentage == {test[1]} & test_type == '{test_type}' & encrypted_type == '{enc_type}' & version == {version}"
#             rows = df.query(query)
#             if rows.empty:
#                 raise ValueError("No Rows: " + str(query))
#             rc = rows.size / len(df.columns)
#             if rc > 1:
#                 raise ValueError("Too many: %s - %s " % (rc, query))
#             #assert row count
#             r = rows

#             ops_sec = rows["OperationThroughput"]
#             p95_lat = rows["Latency95thPercentile"] / 1000000

#             print(ops_sec.iloc[0])
#             print(dir(ops_sec))

#             print(p95_lat)
#             # print(rows)
#             break
#         break
#     break

# #s = set(df["query_column"])
# r


60.06717694643591
['T', '_AXIS_LEN', '_AXIS_ORDERS', '_AXIS_TO_AXIS_NUMBER', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__long__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__'

,Unnamed: 0,Patch ID,Execution,Task Name,Topology,Test,OperationsTotal,DocumentsTotal,SizeTotal,DurationTotal,...,WorkersMin,WorkersMax,ErrorRate,version,encrypted_type,query_column,read_percentage,write_percentage,test_type,topo_type
1,1,63f78a470305b937fcbe9fda,0,medical_workload_credit_cards_100_0,linux-1-node-replSet-fle.2022-11,InsertActor.Crud,1100000,0,0,18312830000000,...,16,16,0,60,encrypted,creditcards,100,0,query,replset


In [46]:
# from IPython import display

sample_html = '<html><body><h3>Hello World</h3><h4>Hello World</h4><h5>Hello World</h5><h6>Hello World</h6></body></html>'

display.HTML(data=sample_html)
